In [1]:
from pyspark import SparkConf, SparkContext
# 스파크 환경 설정 객체 생성
conf = SparkConf().setMaster("local").setAppName("241204_02_MovieLens")
spark =SparkContext(conf=conf).getOrCreate()

24/12/04 14:42:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
user_id : 사용자의 고유 ID
movie_id 영화으의 고유 ID
rating 사용자가 영화에 준 평점
timestamp 평점이 기록된 시간. 

In [ ]:
#별점 1,2,3 별로 카운트 

In [10]:
#데이터 저장소 불러오기 
import os
directory= os.path.join(os.getcwd(), "data")
filename= "u.data"
filepath= os.path.join(directory, filename)

In [22]:
#rdd 생성 
rdd= spark.textFile("file://"+filepath.replace("\\","/"))
rdd

file:///home/lab10/src/data/u.data MapPartitionsRDD[24] at textFile at NativeMethodAccessorImpl.java:0

In [13]:
rdd.take(5)

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596']

In [36]:
def parse(row):
    fields= row.split("\t")
    user_id=fields[0] 
    movie_id=fields[1]
    rating =int(fields[2])
    timestamp=fields[3]
    return user_id, movie_id, rating, timestamp

In [38]:
parse('196\t242\t3\t881250949')

('196', '242', 3, '881250949')

In [47]:
# 데이터를 파싱하여 RDD로 변환  
parsed_rdd=rdd.map(parse)

In [48]:
parsed_rdd.take(5)

[('196', '242', 3, '881250949'),
 ('186', '302', 3, '891717742'),
 ('22', '377', 1, '878887116'),
 ('244', '51', 2, '880606923'),
 ('166', '346', 1, '886397596')]

In [68]:
rating_counts = parsed_rdd.map(lambda x: x[2]) 
rating_counts = rating_counts.countByValue() 

In [70]:
rating_counts

defaultdict(int, {3: 27145, 1: 6110, 2: 11370, 4: 34174, 5: 21201})

In [ ]:
#3점 이상 별점 평균 

In [71]:
filtered_rdd=parsed_rdd.filter(lambda x: x[2] >=3)

In [72]:
#영화 별점 총합과 영화 개수 
sum_count= filtered_rdd.map(lambda x: (x[2],1))
sum_count=sum_count.reduceByKey(lambda a,b:a+b)

In [73]:
#평균 계산 
total_sum = sum_count.map(lambda x:x[0]*x[1]).sum()
total_count=sum_count.map(lambda x:x[1]).sum()

In [76]:
average_rating= total_sum/total_count if total_count >0 else 0

In [77]:
print(f"3점 이상의 평균 평점: {average_rating}")

3점 이상의 평균 평점: 3.927968977217644


In [78]:
spark.stop()
